In [ ]:
from src.data import Dataset, available_datasets
from src import workflow

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

In [ ]:
available_datasets()

## In this example we'll use the f-mnist training data to build an initial model

In [ ]:
ds = Dataset.load('f-mnist_train')

In [ ]:
ds.data.shape

In [ ]:
ds.target.shape

# Step 1: `make_train`

We're going to build an visulization tool for our stylists to use. Let's use the UMAP dimension reduction algorithm for this.

https://umap-learn.readthedocs.io/en/latest/

We can pip install umap via:

`pip install umap-learn`

Time to show off your new reproducible environment skills!

Add `umap-learn` under your pip requirements in the `environment.yml` and run `make requirements`.

In [ ]:
from umap import UMAP

In [ ]:
# We want a 2 dimensional visualization
model = UMAP(n_components=2, random_state=42)

In [ ]:
%%time
model.fit(ds.data)

### Check that UMAP has desired properties

Recall that for an algorithm to work with our built in `make train` and `make predict` scripts, it needs to:
* be a BaseEstimator
* Have a fit method
* Have either a predict or transform method

Let's check that UMAP satisfies these properties.

In [ ]:
from sklearn.base import BaseEstimator

In [ ]:
isinstance(model, BaseEstimator)

In [ ]:
hasattr(model, 'predict')

In [ ]:
hasattr(model, 'transform')

### Exercise: Add UMAP to the available_algorithms

In [ ]:
assert 'UMAP' in workflow.available_algorithms()

### Exercise: Add UMAP on f-mnist_train to the training workflow

In [ ]:
# workflow.add_model()

In [ ]:
workflow.get_model_list()

## TODO: Remove this from the actual tutorial notebook...this is an "answer"

In [ ]:
workflow.add_model(
    dataset_name = 'f-mnist_train',
    algorithm_name = 'UMAP',
    algorithm_params = {'n_components': 2, 'random_state': 42}
)

In [ ]:
!cd .. && make train

In [ ]:
workflow.available_models()

# Step 2: `make_predict`

Here's where we use the model to do a dimension reduction.

In this case, we'll set the flag `is_supervised` to `False`, so that the predction algorithm uses the `tranform` method of UMAP instead of `predict`.

### Exercise: Setup dimension reduction using UMAP on f-mnist_train

In [ ]:
# Fill me in!
# workflow.add_prediction()

In [ ]:
workflow.get_prediction_list()

## TODO: Remove next line from the tutorial notebook...it's an answer!

In [ ]:
workflow.add_prediction(
    dataset_name = 'f-mnist_train',
    model_name = 'UMAP_f-mnist_train_1',
    is_supervised = False
)

In [ ]:
workflow.get_prediction_list()

In [ ]:
!cd .. && make predict

In [ ]:
workflow.available_predictions()

# Step 3: `make summary`

In this case, `make summary` will set up a dataframe that we'll use to make visualizations.



### Prototype what we want to do for our visualization

Let's visualize what happened to our data under dimension reduction. Since we have categories of our data (as the targets), let's use those to colour the data.

In [ ]:
from src.models.predict import load_prediction

In [ ]:
ds_predicted = load_prediction('UMAP_f-mnist_train_1_exp_f-mnist_train_1')

In [ ]:
# Here's the 2d prediction
ds_predicted.data.shape

In [ ]:
# Here are the labels. Use these as color!
ds_predicted.target.shape

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()

for i, axis in enumerate(['x', 'y']):
    df[f'{axis}'] = ds_predicted.data[:, i]

df['target'] = ds_predicted.target.astype(int)
# if we want to do bokeh mouseover  from saved images: add images to the dataset, and add the following line
#df['filename'] = ds_predicted.metdata['filename'] 

This is probably all we need for `make summary`, but let's finish making a picture so that we can get a sense of whether we're missing anything.

In [ ]:
# How many categories do we have? (so we can pick an appropriate colormap)
np.unique(df['target'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set(style='white', rc={'figure.figsize':(12,8)})


In [ ]:
plt.scatter(df['x'], df['y'], c=df['target'], cmap='tab10', s=1)
plt.colorbar();

## TODO: Next step, script the summary

## Aside: How to make images..

In [ ]:
from PIL import Image

In [ ]:
row = ds.data[0]
row.shape

In [ ]:
import numpy as np

w, h = 28, 28
new_row = np.reshape(row, (w, h))
new_row.shape

In [ ]:
img = Image.fromarray(new_row, 'L')
img.show()

In [ ]:
images = []
w, h = 28, 28

for row in ds.data:
    row = np.reshape(row, (w, h))
    img = Image.fromarray(new_row, 'L')
    images.append(img)

In [ ]:
df['images'] = images

In [ ]:
df.head()